In [3]:
import re
import torch
import pandas as pd
from transformers import BertTokenizer, BertModel
from torch.utils.data import Dataset
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader
import torch.nn as nn
from sklearn.metrics import accuracy_score
import ast
import torch.optim as optim
from sentence_transformers import SentenceTransformer
import numpy as np

In [2]:
!pip install sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 5.1 MB/s eta 0:00:00a 0:00:01


In [4]:
df_train = pd.read_csv('/kaggle/input/nn-text-classfication/train.csv')

In [5]:
model_new = SentenceTransformer('all-MiniLM-L6-v2', device='cuda')

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [6]:
df_train.head()

,SampleID,Discussion,Category
0,1,"Without sitting down and doing it manually, yo...",Sports
1,2,All your Search ends with this link.,STEM
2,3,"No, the program you're using is made to be com...",STEM
3,4,Mike Woicik\n\nThe correct answer is: Mike Woi...,Sports
4,5,"No, but not because of why you might think. Wh...",Politics


In [7]:
df_train['Discussion'] = df_train['Discussion'].fillna('No Text')

In [8]:
def replace_dates(text):
    date_pattern = r'\b(\d{1,2}-[A-Za-z]{3}|\b[A-Za-z]+ \d{1,2}(\w{2})?)\b'
    return re.sub(date_pattern, '[DATE]', text)

df_train['Discussion'] = df_train['Discussion'].apply(replace_dates)

In [9]:
# Generate embeddings for the dataset
embeddings_train = model_new.encode(df_train['Discussion'].tolist(), convert_to_tensor=True).cpu().numpy()

# Save embeddings to a file
np.save('news_embeddings_train.npy', embeddings_train)

# Save corresponding labels
df_train['Category'].to_csv('news_labels.csv', index=False)

print("Embeddings and labels saved successfully!")

Batches:   0%|          | 0/781 [00:00<?, ?it/s]

Embeddings and labels saved successfully!


In [10]:
label_mapping = {
    'Politics': 0,
    'Sports': 1,
    'Media': 2,
    'Market & Economy': 3,
    'STEM': 4
}
df_train['Category']=df_train['Category'].map(label_mapping)

In [11]:
df_train['Category']

0        1
1        4
2        4
3        1
4        0
        ..
24984    1
24985    3
24986    3
24987    0
24988    2
Name: Category, Length: 24989, dtype: int64

In [12]:
labels = list(df_train['Category'])

In [13]:
class TextDataset(Dataset):
    def __init__(self, sentences, labels):
        self.sentences = torch.stack([torch.tensor(x, dtype=torch.float32).unsqueeze(0) for x in sentences])
        self.labels = torch.tensor(labels, dtype=torch.long)

    def __len__(self):
        return len(self.sentences)

    def __getitem__(self, idx):
        return self.sentences[idx], self.labels[idx]

In [14]:
X_train, X_val, y_train, y_val = train_test_split(embeddings_train, labels, test_size=0.2, random_state=42)

train_dataset = TextDataset(X_train, y_train)
val_dataset = TextDataset(X_val, y_val)

train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=32, shuffle=False)

In [15]:
for batch in train_dataloader:
    sentences, labels = batch
    print(f"Train batch size: {sentences.size(0)}")
    print(f"Sentence shape: {sentences.shape}")
    print(f"Labels shape: {labels.shape}")
    break

Train batch size: 32
Sentence shape: torch.Size([32, 1, 384])
Labels shape: torch.Size([32])


In [16]:
class TextCNN(nn.Module):
    def __init__(self, input_size, num_classes, kernel_sizes, num_filters, dropout=0.5):
        super(TextCNN, self).__init__()
        self.convs = nn.ModuleList([
            nn.Conv2d(1, num_filters, (k, input_size)) for k in kernel_sizes if k <= 1
        ])
        self.dropout = nn.Dropout(dropout)
        self.fc = nn.Linear(len(kernel_sizes) * num_filters, num_classes)

    def forward(self, x):
        x = x.unsqueeze(1)  # [32, 1, 768]
        
        conv_outputs = [torch.relu(conv(x)).squeeze(3) for conv in self.convs]  # Conv + ReLU
        pooled_outputs = [torch.max(output, dim=2)[0] for output in conv_outputs]  # Max Pooling
        out = torch.cat(pooled_outputs, dim=1) 
        out = self.dropout(out)
        out = self.fc(out)
        return out

In [17]:
# Training function
def train_model(model, train_dataloader, optimizer, criterion, epochs=5, save_path='model.pth', scheduler=None):
    model.train()
    for epoch in range(epochs):
        running_loss = 0.0
        all_preds = []
        all_labels = []
        
        for sentences, labels in train_dataloader:
            sentences = sentences.float().to(device)
            labels = labels.long().to(device)

            optimizer.zero_grad()
            outputs = model(sentences)
            loss = criterion(outputs, labels)
            loss.backward()
            nn.utils.clip_grad_norm_(model.parameters(), max_norm=5.0)  # Gradient clipping
            optimizer.step()

            running_loss += loss.item()
            _, preds = torch.max(outputs, dim=1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
         
        if scheduler:
            scheduler.step()
        
        epoch_loss = running_loss / len(train_dataloader)
        epoch_acc = accuracy_score(all_labels, all_preds) * 100
        print(f"Epoch [{epoch+1}/{epochs}], Loss: {epoch_loss:.4f}, Accuracy: {epoch_acc:.2f}%")

    print("Training complete.")
    torch.save(model.state_dict(), save_path)
    print(f"Model saved to {save_path}")

In [18]:
# Validation function
def val_model(model, val_dataloader):
    model.eval()
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for sentences, labels in val_dataloader:
            sentences = sentences.float().to(device)
            labels = labels.long().to(device)
            outputs = model(sentences)
            _, preds = torch.max(outputs, dim=1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    accuracy = accuracy_score(all_labels, all_preds)
    print(f"Validation Accuracy: {accuracy * 100:.2f}%")
    return accuracy

In [19]:
input_size = 384  # Sentence embedding size
num_classes = len(set(labels))  # Number of unique classes
kernel_sizes = [1]  # Different filter sizes
num_filters = 256  # Number of filters per kernel size
dropout = 0.6
learning_rate = 0.001
epochs = 50

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Initialize TextCNN
model = TextCNN(input_size, num_classes, kernel_sizes, num_filters, dropout).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=learning_rate)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.5)

In [20]:
# Train and validate
train_model(model, train_dataloader, optimizer, criterion, epochs=epochs, save_path='textcnn.pth', scheduler=scheduler)
val_model(model, val_dataloader)

Epoch [1/50], Loss: 1.0515, Accuracy: 70.10%
Epoch [2/50], Loss: 0.7233, Accuracy: 74.54%
Epoch [3/50], Loss: 0.6894, Accuracy: 75.62%
Epoch [4/50], Loss: 0.6696, Accuracy: 75.98%
Epoch [5/50], Loss: 0.6566, Accuracy: 76.48%
Epoch [6/50], Loss: 0.6430, Accuracy: 76.79%
Epoch [7/50], Loss: 0.6352, Accuracy: 77.04%
Epoch [8/50], Loss: 0.6218, Accuracy: 77.37%
Epoch [9/50], Loss: 0.6146, Accuracy: 77.98%
Epoch [10/50], Loss: 0.6008, Accuracy: 78.08%
Epoch [11/50], Loss: 0.5877, Accuracy: 79.06%
Epoch [12/50], Loss: 0.5826, Accuracy: 78.72%
Epoch [13/50], Loss: 0.5761, Accuracy: 79.38%
Epoch [14/50], Loss: 0.5712, Accuracy: 79.33%
Epoch [15/50], Loss: 0.5632, Accuracy: 79.97%
Epoch [16/50], Loss: 0.5585, Accuracy: 80.14%
Epoch [17/50], Loss: 0.5545, Accuracy: 80.21%
Epoch [18/50], Loss: 0.5503, Accuracy: 80.25%
Epoch [19/50], Loss: 0.5477, Accuracy: 80.59%
Epoch [20/50], Loss: 0.5416, Accuracy: 80.69%
Epoch [21/50], Loss: 0.5285, Accuracy: 81.12%
Epoch [22/50], Loss: 0.5278, Accuracy: 81.1

0.7633053221288515

In [21]:
df_test=pd.read_csv('/kaggle/input/nn-text-classfication/test.csv')

In [22]:
df_test['Discussion'] = df_test['Discussion'].fillna('No Text')

In [23]:
def replace_dates(text):
    date_pattern = r'\b(\d{1,2}-[A-Za-z]{3}|\b[A-Za-z]+ \d{1,2}(\w{2})?)\b'
    return re.sub(date_pattern, '[DATE]', text)

df_test['Discussion'] = df_test['Discussion'].apply(replace_dates)

In [24]:
# Generate embeddings for the dataset
embeddings_test = model_new.encode(df_test['Discussion'].tolist(), convert_to_tensor=True).cpu().numpy()

# Save embeddings to a file
np.save('news_embeddings_train.npy', embeddings_test)

print("Embeddings and labels saved successfully!")

Batches:   0%|          | 0/330 [00:00<?, ?it/s]

Embeddings and labels saved successfully!


In [30]:
class CustomDataset(Dataset):
    def __init__(self, sentences, sample_ids):
        # Ensure input embeddings are treated as sequences with one channel
        self.sentences = torch.stack([torch.tensor(x, dtype=torch.float32).unsqueeze(0) for x in sentences])
        self.sample_ids = torch.tensor(sample_ids, dtype=torch.long)

    def __len__(self):
        return len(self.sentences)

    def __getitem__(self, idx):
        return self.sentences[idx], self.sample_ids[idx]

In [31]:
test_dataset = CustomDataset(embeddings_test, df_test['SampleID'])
test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [32]:
save_csv_path = '/kaggle/working/predictions_textcnn1.csv'

In [33]:
def test_model(model, val_dataloader, save_csv_path='predictions.csv', device='cuda'):
    model.eval()  # Set model to evaluation mode
    all_preds = []
    sample_ids = []  # To store sample IDs

    with torch.no_grad():
        for sentences, ids in val_dataloader:  # Extract sentences and IDs from DataLoader
            sentences = sentences.float().to(device)  # Move sentences to GPU (or CPU if needed)
            outputs = model(sentences)
            _, preds = torch.max(outputs, dim=1)

            all_preds.extend(preds.cpu().numpy())  # Move predictions back to CPU
            sample_ids.extend(ids.numpy())  # Collect the sample IDs

    # Save predictions to a CSV file
    predictions_df = pd.DataFrame({
        'SampleID': sample_ids,
        'Category': all_preds
    })
    predictions_df.to_csv(save_csv_path, index=False)
    print(f"Predictions saved to {save_csv_path}")
    
    return predictions_df

In [34]:
test_model(model, test_dataloader, save_csv_path)

Predictions saved to /kaggle/working/predictions_textcnn1.csv


,SampleID,Category
0,1,3
1,2,0
2,3,1
3,4,4
4,5,3
...,...,...
10552,10553,4
10553,10554,3
10554,10555,3
10555,10556,0
